In [1]:
import numpy as np
import ccdproc
from astropy import units as u

In [12]:
import os
dir = '/home/twinkle/Documents/adfosc/2019-06-14-adfosc/109_956/fast_gain/'
from glob import glob
a = sorted(glob(os.path.join(dir,'bias*.fit')))

In [13]:
gain = 1.15 * u.electron / u.adu
readnoise = 10 * u.electron
image=ccdproc.CCDData.read(os.path.join(dir,'mbias.fit'),unit='adu')
image=ccdproc.create_deviation(image, gain=gain, readnoise=readnoise)
image = ccdproc.gain_correct(image, gain)
header=image.header
print(image)

[[ 6299.69970703  6313.5         6306.60009766 ...,  6298.54980469
   6269.79980469  6354.89990234]
 [ 6296.25        6287.04980469  6335.35009766 ...,  6270.94970703
   6305.44970703  6323.85009766]
 [ 6313.5         6307.75        6307.75       ...,  6283.60009766
   6358.35009766  6299.69970703]
 ..., 
 [ 6238.75        6305.44970703  6260.60009766 ...,  6231.85009766
   6230.69970703  6258.29980469]
 [ 6292.79980469  6304.29980469  6315.79980469 ...,  6290.5         6339.94970703
   6292.79980469]
 [ 6314.64990234  6280.14990234  6284.75       ...,  6312.35009766
   6352.60009766  6188.14990234]]


In [11]:
gain = 1.15 * u.electron / u.adu
readnoise = 10 * u.electron
biaslist = []
for i in range (0,len(a)):
    b=a[i]
    data= ccdproc.CCDData.read(b,unit='adu')
    #data = ccdproc.create_deviation(data, gain=gain, readnoise=readnoise)
    #data= data-(data.uncertainty.array)
    biaslist.append(data)
combiner = ccdproc.Combiner(biaslist)
masterbias = combiner.median_combine()   
masterbias.write('masterbias.fit', overwrite=True)
mbias=ccdproc.CCDData.read('masterbias.fit',unit='adu')
masterbias.meta=image.meta
print(masterbias)


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
[[ 5478.  5490.  5484. ...,  5477.  5452.  5526.]
 [ 5475.  5467.  5509. ...,  5453.  5483.  5499.]
 [ 5490.  5485.  5485. ...,  5464.  5529.  5478.]
 ..., 
 [ 5425.  5483.  5444. ...,  5419.  5418.  5442.]
 [ 5472.  5482.  5492. ...,  5470.  5513.  5472.]
 [ 5491.  5461.  5465. ...,  5489.  5524.  5381.]]


In [ ]:
c=sorted(glob(os.path.join(dir,'blank*.fit')))
flatlist = []
for j in range(0,len(c)):
    d=c[j]
    flat=ccdproc.CCDData.read(d,unit='adu')
    flat=ccdproc.subtract_bias(flat,masterbias)
    flatlist.append(flat)
combiner = ccdproc.Combiner(flatlist)
masterflat = combiner.median_combine()
masterflat.write('masterflat.fit', overwrite=True)
mflat=ccdproc.CCDData.read('masterflat.fit',unit='adu')
masterflat.meta=image.meta

    
    

   
    
  
    

In [ ]:
# remove mbias and mflat
e = sorted(glob(os.path.join(dir,'109*.fit')))
for j in range(0,len(c)):
    f=e[j]
    image=ccdproc.CCDData.read(f,unit='adu')
    bias_subtracted = ccdproc.subtract_bias(image, masterbias)
    reduced_image = ccdproc.flat_correct(bias_subtracted, masterflat)
print(reduced_image)    

In [ ]:
cr_cleaned = ccdproc.cosmicray_lacosmic(gain_corrected, sigclip=5)
